<a href="https://colab.research.google.com/github/elsyanares/Deep-Learning-Computational-Intelligence/blob/main/MidtermTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Elsya Nareswari
2106702535

In [1]:
!pip install torch torchvision torchaudio
!pip install opencv-python-headless
!git clone https://github.com/ultralytics/yolov5  # Clone YOLOv5 repo
%cd yolov5
!pip install -r requirements.txt  # Install YOLOv5 dependencies


Cloning into 'yolov5'...
remote: Enumerating objects: 17022, done.
remote: Total 17022 (delta 0), reused 0 (delta 0), pack-reused 17022 (from 1)
Receiving objects: 100% (17022/17022), 15.62 MiB | 8.44 MiB/s, done.
Resolving deltas: 100% (11694/11694), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.0/879.0 kB 53.8 MB/s eta 0:00:00


In [2]:
import torch
import cv2
import pandas as pd
import json
from google.colab import files


In [3]:
# Load the YOLOv5 model (choose the appropriate model version)
model = torch.hub.load('ultralytics/yolov5', 'yolov5m', pretrained=True)  # Use 'yolov5m' model


/usr/local/lib/python3.10/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2024-11-2 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

100%|██████████| 40.8M/40.8M [00:01<00:00, 24.1MB/s]

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients, 48.9 GFLOPs
Adding AutoShape... 


In [4]:
# Path to input video
input_video = "/content/british_highway_traffic.mp4"
output_video = "processed_traffic.mp4"

# Open video for reading
cap = cv2.VideoCapture(input_video)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define codec and output video writer
out = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))


In [5]:
# List to store structured detection results
structured_data = []
frame_count = 0

# Process each frame in the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Stop if the video is finished

    frame_count += 1

    # Run inference on the frame
    results = model(frame, size=640)  # Resize input for YOLOv5

    # Process each detection in the frame
    for *xyxy, conf, cls in results.xyxy[0]:
        x1, y1, x2, y2 = map(int, xyxy)  # Absolute bounding box

        # Draw bounding box and label on the frame
        label = f"{model.names[int(cls)]} {conf:.2f}"
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Normalize coordinates to the range [0, 1]
        x1_norm = x1 / width
        y1_norm = y1 / height
        x2_norm = x2 / width
        y2_norm = y2 / height

        # Save structured data for tracking
        structured_data.append({
            'frame': frame_count,
            'label': model.names[int(cls)],  # Class name
            'confidence': float(conf),  # Confidence score
            'x1_norm': x1_norm, 'y1_norm': y1_norm,
            'x2_norm': x2_norm, 'y2_norm': y2_norm
        })

    # Write the processed frame to the output video
    out.write(frame)


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
WARNING ⚠️ NMS time limit 0.550s exceeded
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/co

In [6]:
# Release resources
cap.release()
out.release()
print("Video processing completed. Saved as 'processed_traffic.mp4'")


Video processing completed. Saved as 'processed_traffic.mp4'


In [7]:
# Download the output files
files.download('processed_traffic.mp4')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>